# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [102]:
# import libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier

import pickle

In [83]:
# load data from database
engine = create_engine('sqlite:///message_data.db')
df = pd.read_sql_table('message_data', engine)
X = df['message']
Y = df.iloc[:, 4:]

### 2. Write a tokenization function to process your text data

In [84]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = []
    for token in tokens:
        if token not in stopwords.words('english'):
            clean_tok = lemmatizer.lemmatize(token).lower().strip()
            clean_tokens.append(clean_tok)
    
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [85]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(), n_jobs = -1))
])

In [86]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x00000168D1D29820>)),
  ('tfidf', TfidfTransformer()),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(), n_jobs=-1))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x00000168D1D29820>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier(), n_jobs=-1),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfid

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [87]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x00000168D1D29820>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(),
                                       n_jobs=-1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [88]:
y_pred = pipeline.predict(X_test)
y_pred = pd.DataFrame(y_pred)

In [89]:
for i, col in enumerate(Y.columns):
    print('Category: ', col)
    print(classification_report(y_test.iloc[:, i], y_pred.iloc[:, i]))

Category:  related
              precision    recall  f1-score   support

           0       0.73      0.36      0.48      1528
           1       0.82      0.96      0.89      4968
           2       1.00      0.14      0.24        58

    accuracy                           0.81      6554
   macro avg       0.85      0.48      0.54      6554
weighted avg       0.80      0.81      0.79      6554

Category:  request
              precision    recall  f1-score   support

           0       0.89      0.98      0.94      5405
           1       0.84      0.45      0.59      1149

    accuracy                           0.89      6554
   macro avg       0.87      0.72      0.76      6554
weighted avg       0.88      0.89      0.87      6554

Category:  offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6527
           1       0.00      0.00      0.00        27

    accuracy                           1.00      6554
   macro avg       

C:\Users\jrsfe\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6257
           1       0.88      0.18      0.29       297

    accuracy                           0.96      6554
   macro avg       0.92      0.59      0.64      6554
weighted avg       0.96      0.96      0.95      6554

Category:  other_aid
              precision    recall  f1-score   support

           0       0.87      1.00      0.93      5698
           1       0.71      0.04      0.08       856

    accuracy                           0.87      6554
   macro avg       0.79      0.52      0.50      6554
weighted avg       0.85      0.87      0.82      6554

Category:  infrastructure_related
              precision    recall  f1-score   support

           0       0.94      1.00      0.97      6135
           1       0.00      0.00      0.00       419

    accuracy                           0.94      6554
   macro avg       0.47      0.50      0.48      6554
weighted avg       

### 6. Improve your model
Use grid search to find better parameters. 

In [96]:
# Takes forever
parameters = {
    'tfidf__use_idf': (True, False)
}

cv = GridSearchCV(pipeline, parameters, verbose = 3)
cv.fit(X_train, y_train)
cv.best_params_

Fitting 5 folds for each of 2 candidates, totalling 10 fits
[CV] tfidf__use_idf=True .............................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ................. tfidf__use_idf=True, score=0.256, total= 3.5min
[CV] tfidf__use_idf=True .............................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  3.5min remaining:    0.0s


[CV] ................. tfidf__use_idf=True, score=0.244, total= 3.5min
[CV] tfidf__use_idf=True .............................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  6.9min remaining:    0.0s


[CV] ................. tfidf__use_idf=True, score=0.259, total= 3.5min
[CV] tfidf__use_idf=True .............................................
[CV] ................. tfidf__use_idf=True, score=0.265, total= 3.5min
[CV] tfidf__use_idf=True .............................................
[CV] ................. tfidf__use_idf=True, score=0.240, total= 3.5min
[CV] tfidf__use_idf=False ............................................
[CV] ................ tfidf__use_idf=False, score=0.258, total= 3.6min
[CV] tfidf__use_idf=False ............................................
[CV] ................ tfidf__use_idf=False, score=0.242, total= 3.5min
[CV] tfidf__use_idf=False ............................................
[CV] ................ tfidf__use_idf=False, score=0.257, total= 3.5min
[CV] tfidf__use_idf=False ............................................
[CV] ................ tfidf__use_idf=False, score=0.256, total= 3.4min
[CV] tfidf__use_idf=False ............................................
[CV] .

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed: 34.8min finished


{'tfidf__use_idf': True}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [97]:
y_pred = cv.predict(X_test)
y_pred = pd.DataFrame(y_pred)

In [98]:
for i, col in enumerate(Y.columns):
    print('Category: ', col)
    print(classification_report(y_test.iloc[:, i], y_pred.iloc[:, i]))

Category:  related
              precision    recall  f1-score   support

           0       0.73      0.35      0.47      1528
           1       0.82      0.96      0.89      4968
           2       0.80      0.07      0.13        58

    accuracy                           0.81      6554
   macro avg       0.78      0.46      0.50      6554
weighted avg       0.80      0.81      0.78      6554

Category:  request
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      5405
           1       0.83      0.46      0.59      1149

    accuracy                           0.89      6554
   macro avg       0.86      0.72      0.77      6554
weighted avg       0.88      0.89      0.88      6554

Category:  offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6527
           1       0.00      0.00      0.00        27

    accuracy                           1.00      6554
   macro avg       

C:\Users\jrsfe\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.94      1.00      0.97      6135
           1       0.00      0.00      0.00       419

    accuracy                           0.94      6554
   macro avg       0.47      0.50      0.48      6554
weighted avg       0.88      0.94      0.91      6554

Category:  transport
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6264
           1       0.65      0.08      0.15       290

    accuracy                           0.96      6554
   macro avg       0.80      0.54      0.56      6554
weighted avg       0.95      0.96      0.94      6554

Category:  buildings
              precision    recall  f1-score   support

           0       0.95      1.00      0.98      6215
           1       0.81      0.11      0.20       339

    accuracy                           0.95      6554
   macro avg       0.88      0.56      0.59      6554
weighted avg       0.95      0.9

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [106]:
pipeline2 = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(AdaBoostClassifier()))
])

In [107]:
pipeline2.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x00000168D1D29820>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=AdaBoostClassifier()))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x00000168D1D29820>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=AdaBoostClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,
 'tfidf_

In [108]:
# parameters = {
#     'tfidf__use_idf': (True, False)
# }

# cv2 = GridSearchCV(pipeline, parameters)
pipeline2.fit(X_train, y_train)
# cv2.best_params_

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x00000168D1D29820>)),
                ('tfidf', TfidfTransformer()),
                ('clf', MultiOutputClassifier(estimator=AdaBoostClassifier()))])

In [109]:
y_pred = pipeline2.predict(X_test)
y_pred = pd.DataFrame(y_pred)

In [110]:
for i, col in enumerate(Y.columns):
    print('Category: ', col)
    print(classification_report(y_test.iloc[:, i], y_pred.iloc[:, i]))

Category:  related
              precision    recall  f1-score   support

           0       0.50      0.07      0.12      1528
           1       0.77      0.98      0.86      4968
           2       0.57      0.21      0.30        58

    accuracy                           0.76      6554
   macro avg       0.61      0.42      0.43      6554
weighted avg       0.70      0.76      0.68      6554

Category:  request
              precision    recall  f1-score   support

           0       0.90      0.97      0.93      5405
           1       0.75      0.50      0.60      1149

    accuracy                           0.88      6554
   macro avg       0.83      0.73      0.77      6554
weighted avg       0.88      0.88      0.87      6554

Category:  offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6527
           1       0.00      0.00      0.00        27

    accuracy                           0.99      6554
   macro avg       

              precision    recall  f1-score   support

           0       0.95      0.99      0.97      6203
           1       0.52      0.14      0.22       351

    accuracy                           0.95      6554
   macro avg       0.74      0.56      0.59      6554
weighted avg       0.93      0.95      0.93      6554

Category:  direct_report
              precision    recall  f1-score   support

           0       0.87      0.96      0.91      5271
           1       0.70      0.42      0.53      1283

    accuracy                           0.85      6554
   macro avg       0.79      0.69      0.72      6554
weighted avg       0.84      0.85      0.84      6554



### 9. Export your model as a pickle file

In [ ]:
with open('ML_Model.pkl', 'wb') as f:
    pickle.dump(cv, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.